<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# Construction des caractéristiques ou *features*

Les données textuelles ne peuvent pas être utilisés directment dans les différents algorithmes de d'apprentissage statistiques. Nous allons voir dans ce tutoriel plusieurs technique permettant de traduires les données textuelles sous formes de vecteur numérique : 


Nombresue fonction de vectorisation présente dans scikit-learn :

* `One-Hot-Encoder`
* `Tf-Idf`
* `Hashing`

Word Embedding dans la librairies gensim :

* `Word2Vec`


## Téléchargement des librairies

In [1]:
#Importation des librairies utilisées
import time
import pandas as pd
import numpy as np
import collections
import itertools
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

## Téléchargement des données

In [2]:
data_test_clean_stem = pd.read_csv("data/cdiscount_test_clean_stem.csv").fillna("")
data_train_clean_stem = pd.read_csv("data/cdiscount_train_clean_stem.csv").fillna("")

On créé un dossier dans lequel nous allons sauvegarder les DataFrame constitués des features que l'on va construire dans ce notebook 

In [3]:
DATA_OUTPUT_DIR = "data/features"
if not(os.path.isdir("data/features")):
    os.mkdir("data/features")

Dans un premier temps on considère seulement la colonne *Description* de nos `DataFrame`

In [4]:
train_array = data_train_clean_stem["Description"].values
test_array = data_test_clean_stem["Description"].values

## Vectorisaton

### One-Hot-Encoding

* **One-Hot-Encoding** ...

possibilité de ngram/Decrire les paramètres

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

extr_cv = CountVectorizer(binary=False)
data_train_OHE = extr_cv.fit_transform(train_array)
data_train_OHE

<95000x56624 sparse matrix of type '<class 'numpy.int64'>'
	with 1119190 stored elements in Compressed Sparse Row format>

In [6]:
vocabulary = extr_cv.get_feature_names()
N_vocabulary = len(vocabulary)
N_vocabulary

56624

Affiche la première ligne

In [7]:
ir = 0

rw = data_train_OHE.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vocabulary[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","weight"])


batter pavilion mah ion batter pavilion mah ioncaracterist voir present


,indices,token,weight
0,41187,present,1
1,54416,voir,1
2,26713,ioncaracterist,1
3,26704,ion,1
4,31648,mah,2
5,38751,pavilion,2
6,4784,batter,2


In [8]:
data_test_OHE = extr_cv.transform(test_array)
data_test_OHE

<5000x56624 sparse matrix of type '<class 'numpy.int64'>'
	with 56875 stored elements in Compressed Sparse Row format>

In [9]:
ir = 5

rw = data_test_OHE.getrow(ir)
print(test_array[ir])
pd.DataFrame([(v, vocabulary[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","weight"])


sel guerand epic verrin sel guerand epic verrin composit ingredient sel guerand kashmir masal


,indices,token,weight
0,11416,composit,1
1,18243,epic,2
2,23630,guerand,3
3,26214,ingredient,1
4,32339,masal,1
5,46174,sel,3
6,53740,verrin,2


### TF-IDF¶

* **TF-IDF**. Le TF-IDF permet de faire ressortir l'importance relative de chaque mot $m$ (ou couples de mots consécutifs) dans un texte-produit ou un descriptif $d$, par rapport à la liste entière des produits. La fonction $TF(m,d)$ compte le nombre d'occurences du mot $m$ dans le descriptif $d$. La fonction $IDF(m)$ mesure l'importance du terme dans l'ensemble des documents ou descriptifs en donnant plus de poids aux termes les moins fréquents car considérés comme les plus discriminants (motivation analogue à celle de la métrique du chi2 en anamlyse des correspondance). $IDF(m,l)=\log\frac{D}{f(m)}$ où $D$ est le nombre de documents, la taille de l'échantillon d'apprentissage, et $f(m)$ le nombre de documents ou descriptifs contenant le mot $m$. La nouvelle variable ou *features* est $V_m(l)=TF(m,l)\times IDF(m,l)$.

* Comme pour les transformations des variables quantitatives (centrage, réduction), la même transformation c'est-à-dire les mêmes pondérations, est calculée sur l'achantillon d'apprentissage et appliquée à celui de test.

On utiliser la fonction TfidfVectorizer qui permet de parser également le texte

On fixe le paramètre `norm` = False pour rendre les résultats plus explicite 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer( ngram_range=(1,1), norm = False)
data_train_TFIDF = vec.fit_transform(train_array)
data_train_TFIDF

<95000x56624 sparse matrix of type '<class 'numpy.float64'>'
	with 1119190 stored elements in Compressed Sparse Row format>

In [11]:
vocabulary = vec.get_feature_names()
N_vocabulary = len(vocabulary)
N_vocabulary

56624

In [12]:
ir = 0

rw = data_train_TFIDF.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vocabulary[v], vec.idf_[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","idf","weight"])


batter pavilion mah ion batter pavilion mah ioncaracterist voir present


,indices,token,idf,weight
0,4784,batter,3.599301,7.198601
1,38751,pavilion,4.742069,9.484137
2,31648,mah,3.652676,7.305352
3,26704,ion,4.199600,4.199600
4,26713,ioncaracterist,7.081745,7.081745
5,54416,voir,1.334262,1.334262
6,41187,present,1.340531,1.340531


Commentez, Comment evolue les poids en changeant les paramètre smooth idf, sublinear_tf?

En cchangean le nombre de ngram?

On applique maintenant le `vectorizer` sur le jeu de données de test

In [13]:
data_test_TFIDF = vec.transform(test_array)
data_test_TFIDF

<5000x56624 sparse matrix of type '<class 'numpy.float64'>'
	with 56875 stored elements in Compressed Sparse Row format>

Si le tf est recalculé pour chaque ligne, le même idf est utilisé

In [14]:
ir = 5

rw = data_test_TFIDF.getrow(ir)
print(test_array[ir])
pd.DataFrame([(v, vocabulary[v], vec.idf_[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","idf","weight"])


sel guerand epic verrin sel guerand epic verrin composit ingredient sel guerand kashmir masal


,indices,token,idf,weight
0,53740,verrin,10.515733,21.031465
1,46174,sel,7.594108,22.782325
2,32339,masal,11.768496,11.768496
3,26214,ingredient,8.654980,8.654980
4,23630,guerand,11.363030,34.089091
5,18243,epic,8.654980,17.309960
6,11416,composit,5.512745,5.512745


### Hashing

* **Hashage**. Il permet de réduire l'espace des variables (taille du dictionnaire) en un nombre limité et fixé a priori `n_hash` de caractéristiques. Il repose sur la définition d'une fonction de hashage, $h$ qui à un indice $j$ défini dans l'espace des entiers naturels, renvoie un indice $i=h(j)$ dans dans l'espace réduit (1 à n_hash) des caractéristiques. Ainsi le poids de l'indice $i$, du nouvel espace, est l'association de tous les poids d'indice $j$ tels que $i=h(j)$ de l'espace originale. Ici, les poids sont associés d'après la méthode décrite par Weinberger et al. (2009).

N.B. $h$ n'est pas généré aléatoirement. Ainsi pour un même fichier d'apprentissage (ou de test) et pour un même entier n_hash, le résultat de la fonction de hashage est identique

Take a Dictionnary as an input

In [15]:
from sklearn.feature_extraction import FeatureHasher
nb_hash = 300

feathash = FeatureHasher(nb_hash)
train_dict_array  = map(lambda x : collections.Counter(x.split(" ")), train_array)
data_train_hash = feathash.fit_transform(train_dict_array)

In [16]:
data_train_hash


<95000x300 sparse matrix of type '<class 'numpy.float64'>'
	with 1093403 stored elements in Compressed Sparse Row format>

In [17]:

ir = 0

rw = data_train_hash.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","weight"])


batter pavilion mah ion batter pavilion mah ioncaracterist voir present


,indices,weight
0,14,2.0
1,36,1.0
2,57,2.0
3,175,1.0
4,204,2.0
5,220,-1.0
6,274,1.0


Reduit la taille de la matrice, MAIS pas de fonction inverse transform. 
Resultat beaucoup moin facilement compréhensible
Efficace que si vraient beaucoup de ligne (>100.000)

Il est ensuite possible de passer la nouvelle matrice *hasher* (`data_train_hash`) directement dans un algorithme d'apprentissage, ou bien de combiner ces poids avec une fonction de TFIDF

Cette fois la fonction `TFIDFTransformer`est utilisé, ne considère pas les string, mais les tf comme les poids présent

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer

vec =  TfidfTransformer(norm = False)
data_train_HTfidf = vec.fit_transform(data_train_hash)
data_train_HTfidf

<95000x300 sparse matrix of type '<class 'numpy.float64'>'
	with 1085031 stored elements in Compressed Sparse Row format>

In [19]:
ir = 0

rw = data_train_HTfidf.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vec.idf_[v], k)  for k,v in zip(rw.data, rw.indices)], columns=["indices","idf_","weight"])


batter pavilion mah ion batter pavilion mah ioncaracterist voir present


,indices,idf_,weight
0,274,1.309974,1.309974
1,220,1.325886,-1.325886
2,204,3.178517,6.357033
3,175,3.764965,3.764965
4,57,3.506969,7.013938
5,36,5.266455,5.266455
6,14,3.283309,6.566618


### Save Vectorize Vector

De nombreux paramètres a regler entraine donc un un très grand nombre de combinaison.

ici nous créer seulement par défault 4 jeu de données avec count et TFIDF chacun avec et sans hashin de taille 300.

A vous de jouer sur d'autre paramètre nombre de gram, taille de la fonction de hashing, colonne pris en compte ... pour générer d'autre dataset et tester leur influence

### Fonction de Vectorisation

on créé deux fonctions `vectorizer_train` and `apply_vectorizer` afin de générer automatiquement différent dataframe d'apprentissage et de test vectorisé.

In [20]:
def vectorizer_train(df, columns=['Description'], nb_hash=None, nb_gram = 1, vectorizer = "tfidf" , binary = False):
    
    data_array = [" ".join(line) for line in df[columns].values]
    
    # Hashage
    if nb_hash is None:
        feathash = None
        if vectorizer == "tfidf":
            vec = TfidfVectorizer(ngram_range=(1,nb_gram))
            data_vec = vec.fit_transform(data_array)
        else:
            vec = CountVectorizer(binary=binary)
            data_vec = vec.fit_transform(data_array)
    else:
        data_dic_array = [collections.Counter(line.split(" ")) for line in data_array]
        feathash = FeatureHasher(nb_hash)
        data_hash = feathash.fit_transform(data_dic_array)
        
        if vectorizer=="tfidf":
            vec =  TfidfTransformer()
            data_vec =  vec.fit_transform(data_hash)
        else:
            vec = None
            data_vec = data_hash

    return vec, feathash, data_vec



def apply_vectorizer(df, vec, feathash, columns =['Description', 'Libelle', 'Marque']):
    
    data_array = [" ".join(line) for line in df[columns].values]
    
    #Hashage
    if feathash is None:
        data_hash = data_array
    else:
        data_dic_array = [collections.Counter(line.split(" ")) for line in data_array]
        data_hash = feathash.transform(data_dic_array)
    
    if vec is None:
        data_vec = data_hash
    else:
        data_vec = vec.transform(data_hash)
    return data_vec


In [21]:
parameters = [[None, "count"],
              [300, "count"],
              [10000,"count"],
              [None, "tfidf"],
              [300, "tfidf"],
             [10000,"tfidf"]]

from scipy import sparse

for nb_hash, vectorizer in parameters:
    ts = time.time()
    vec, feathash, data_train_vec = vectorizer_train(data_train_clean_stem, nb_hash=nb_hash, vectorizer = vectorizer)
    data_test_vec = apply_vectorizer(data_test_clean_stem, vec, feathash)
    te = time.time()
    
    print("nb_hash : " + str(nb_hash) + ", vectorizer : " + str(vectorizer))
    print("Runing time for vectorization : %.1f seconds" %(te-ts))
    print("Train shape : " + str(data_train_vec.shape))
    print("Test shape : " + str(data_test_vec.shape))

    
    sparse.save_npz(DATA_OUTPUT_DIR +"/vec_train_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer), data_train_vec)
    sparse.save_npz(DATA_OUTPUT_DIR +"/vec_test_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer), data_test_vec)
    

nb_hash : None, vectorizer : count
Runing time for vectorization : 1.3 seconds
Train shape : (95000, 56624)
Test shape : (5000, 56624)
nb_hash : 300, vectorizer : count
Runing time for vectorization : 1.1 seconds
Train shape : (95000, 300)
Test shape : (5000, 300)
nb_hash : 10000, vectorizer : count
Runing time for vectorization : 1.1 seconds
Train shape : (95000, 10000)
Test shape : (5000, 10000)
nb_hash : None, vectorizer : tfidf
Runing time for vectorization : 1.7 seconds
Train shape : (95000, 56624)
Test shape : (5000, 56624)
nb_hash : 300, vectorizer : tfidf
Runing time for vectorization : 1.1 seconds
Train shape : (95000, 300)
Test shape : (5000, 300)
nb_hash : 10000, vectorizer : tfidf
Runing time for vectorization : 1.0 seconds
Train shape : (95000, 10000)
Test shape : (5000, 10000)


## Word2Vec

In [22]:
import gensim

import nltk 
stemmer=nltk.stem.SnowballStemmer('french')

### Build Word2Vec model

In [23]:
train_array_token = [line.split(" ") for line in train_array]
test_array_token = [line.split(" ") for line in test_array]

In [24]:
Features_dimension = 300
sg = 1
hs = 0
negative = 10
X = train_array_token
N_train = len(X)
#sg (int {1, 0}) – Defines the training algorithm. If 1, skip-gram is employed; otherwise, CBOW is used.


#hs (int {1,0}) – If 1, hierarchical softmax will be used for model training. 
#If set to 0, and negative is non-zero, negative sampling will be used.

#negative (int) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). 
#If set to 0, no negative sampling is used

Utilisation d'un modèle appris sur un corpus de Wikipedia en français. Disponible sur le dépôt de 
[Kyubyong Park](https://github.com/Kyubyong/wordvectors).

In [25]:
model_dic = {}
for sg in [0,1]:
    print("Start learning Word2Vec learning")
    print("Params are : Fdim_%d_sg_%d_hs_%d_negative_%d_model" %(Features_dimension, sg, hs, negative))
    ts = time.time()
    model = gensim.models.Word2Vec(X, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
    te = time.time()
    t_learning = te-ts

    # Metadata
    N_vocab, feature_dim = model.wv.vectors.shape
    metadata = {"learning_time" : t_learning, "vocab_size" : N_vocab, "sg" : sg, "negative": negative, "hs":hs}
    print(metadata)
    
    model_name = "skip-gram" if sg==1 else "CBOW"
    model_dic.update({model_name : model})

model_online_dir = "data/fr/fr.bin"
model = gensim.models.Word2Vec.load(model_online_dir)
model_dic.update({"online" : model})

Start learning Word2Vec learning
Params are : Fdim_300_sg_0_hs_0_negative_10_model
{'learning_time': 15.90987777709961, 'vocab_size': 56625, 'sg': 0, 'negative': 10, 'hs': 0}
Start learning Word2Vec learning
Params are : Fdim_300_sg_1_hs_0_negative_10_model
{'learning_time': 54.87539887428284, 'vocab_size': 56625, 'sg': 1, 'negative': 10, 'hs': 0}


### Most similar world
Résultats pertinent seulement si N_train=1.000.000, 100.000 trop peu.

In [26]:
model.predict_output_word(["homm"])

In [27]:
term="homme"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : homme


,CBOW,skip-gram,online
0,femm,unisex,aventurier
1,unisex,arman,aristocrate
2,norway,adid,garçon
3,geographical,emporio,vieillard
4,ingersoll,garcon,délinquant
5,arman,stuhrling,écrivain
6,navy,chronograph,politicien
7,emporio,men,escroc
8,pantalon,adh,orateur
9,vest,chrono,érudit


In [28]:
term="femme"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : femme


,CBOW,skip-gram,online
0,homm,unisex,servante
1,unisex,femmecollect,mère
2,fashion,femmecouleur,fille
3,ballerin,fashion,belle-mère
4,sandal,arman,fiancée
5,srkz,noircomposit,compagne
6,bicolor,versac,sœur
7,escarpin,bred,prostituée
8,bottin,dkny,nourrice
9,arman,cendriyon,demi-sœur


In [29]:
term="xbox"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : xbox


,CBOW,skip-gram,online
0,travel,microsoft,playstation
1,octan,wii,gamecube
2,erupt,playstat,dreamcast
3,krakau,controll,psp
4,embedded,scrib,wii
5,subliminal,unleashed,neo-geo
6,hauck,cruz,mega
7,robocop,kobo,advance
8,qnt,storio,famicom
9,cobby,scuf,nintendo


In [30]:
terms_positif = ["femme","roi"]
terms_negatif = ["homme"]

df_ = []
columns = []
for model_name, model in model_dic.items():
    token_positif = [stemmer.stem(term) if "online"!=model_name else term for term in terms_positif]
    token_negativ = [stemmer.stem(term) if "online"!=model_name else term for term in terms_negatif]
    
    mpow = model.wv.most_similar(positive=token_positif, negative=token_negativ)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,skip-gram,online
0,lapin,coquin,reine
1,jungl,sorcier,belle-mère
2,halloween,ruin,brunehilde
3,romant,poupon,régente
4,pap,paon,reine-mère
5,amour,redecouvr,belle-sœur
6,eleph,bstick,veuve
7,lion,tendress,mère
8,canard,fev,demi-sœur
9,poisson,jungl,nièce


In [31]:
terms_positif = ["espagne","paris"]
terms_negatif = ["france"]

df_ = []
columns = []
for model_name, model in model_dic.items():
    token_positif = [stemmer.stem(term) if "online"!=model_name else term for term in terms_positif]
    token_negativ = [stemmer.stem(term) if "online"!=model_name else term for term in terms_negatif]
    
    mpow = model.wv.most_similar(positive=token_positif, negative=token_negativ)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,skip-gram,online
0,montan,arizon,ségovie
1,gourmandis,autrich,madrid
2,flex,grec,alicante
3,alabam,vilain,aix-en-provence
4,neon,colomb,ancône
5,liberty,entrecrois,séville
6,lichtertanz,ric,malaga
7,canc,nigeri,andalousie
8,cub,manil,alger
9,handbag,cancun,avignon


### Predict output word

In [32]:
terms = ["voir","la"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,skip-gram,online
0,infiniteshopping,referent,ci-dessous
1,present,appartient,ci-dessus
2,neuv,lign,ci-après
3,vot,main,l'article
4,sou,sac,infra
5,presen,reduit,paragraphe
6,desig,prix,bibliographie
7,pres,if,liste
8,presentat,couleur,ci-contre
9,cha,photo,section


In [33]:
terms = ["coque",'pour',"samsung"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,skip-gram,online
0,galaxy,ace,acier
1,ace,galaxy,aluminium
2,express,trend,coque
3,not,cor,former
4,stuff,express,béton
5,trend,not,métallique
6,ref,franc,protéger
7,xcov,plai,remplacer
8,plai,drapeau,fabriquer
9,soupl,soupl,conçue


In [34]:
terms = ["homme"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,skip-gram,online
0,montr,chronograph,d'affaires
1,genr,stuhrling,vieil
2,ditl,hood,d’affaires
3,geographical,chemis,politique
4,field,tissot,d'état
5,chronograph,victorinox,d'église
6,ferry,montr,cet
7,box,bruc,jeune
8,vet,swiss,néandertal
9,boxer,anarchy,d’état


In [35]:
terms = ["femme"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,skip-gram,online
0,montr,montr,sa
1,ditl,cendriyon,jeune
2,cendriyon,dkny,d'affaires
3,chaussur,bred,mariée
4,genr,bulov,enceinte
5,vet,revon,nue
6,bagu,genev,adultère
7,tong,hubert,assassinée
8,hilfig,lublin,fatale
9,giorgio,skagen,homme


### Build Features

#### Our Build Model

In [36]:
data_test_clean = pd.read_csv("data/cdiscount_test_clean.csv").fillna("")
data_train_clean = pd.read_csv("data/cdiscount_train_clean.csv").fillna("")

train_array_token_wstem = [line.split(" ") for line in data_train_clean["Description"].values]
test_array_token_wstem = [line.split(" ") for line in data_test_clean["Description"].values]

In [37]:
def get_features_mean(lines):
    features = [MODEL[x] for x  in lines if x in MODEL]
    if features == []:   
        fm =np.ones(F_SIZE)
    else :
        fm = np.mean(features,axis=0)
    return fm

def get_matrix_features_means(X):
    X_embedded_ = list(map(get_features_mean, X))
    X_embedded = np.vstack(X_embedded_)
    return X_embedded



In [38]:
for model_name in ["CBOW","skip-gram", "online"]:
    
    if "online" == model_name:
        X_train = train_array_token_wstem
        X_test = test_array_token_wstem
    else:
        X_train = train_array_token
        X_test = test_array_token
    
    model = model_dic[model_name]
    MODEL = model
    F_SIZE = Features_dimension

    ts = time.time()
    X_embedded_train = get_matrix_features_means(X_train)
    te = time.time()
    t_train = te-ts
    #np.save(embedded_train_dir, X_embedded_train)
    print("Time conversion : %d seconds"%t_train)
    print("Shape Matrix : (%d,%d)"%X_embedded_train.shape)
    np.save(DATA_OUTPUT_DIR +"/embedded_train_nb_hash_"+model_name, X_embedded_train)

    
    ts = time.time()
    X_embedded_test = get_matrix_features_means(X_test)
    te = time.time()
    t_test = te-ts
    #np.save(embedded_test_dir, X_embedded_test)
    print("Time conversion : %d seconds"%t_test)
    print("Shape Matrix : (%d,%d)"%X_embedded_test.shape)
    np.save(DATA_OUTPUT_DIR +"/embedded_test_nb_hash_"+model_name, X_embedded_test)
    
    metadata = {"t_train" : t_train, "t_test" : t_test, "sg":sg}
    print(metadata)

Time conversion : 10 seconds
Shape Matrix : (95000,300)
Time conversion : 0 seconds
Shape Matrix : (5000,300)
{'t_train': 10.871941328048706, 't_test': 0.5806784629821777, 'sg': 1}
Time conversion : 11 seconds
Shape Matrix : (95000,300)
Time conversion : 0 seconds
Shape Matrix : (5000,300)
{'t_train': 11.681430339813232, 't_test': 0.5803375244140625, 'sg': 1}
Time conversion : 9 seconds
Shape Matrix : (95000,300)
Time conversion : 0 seconds
Shape Matrix : (5000,300)
{'t_train': 9.758267164230347, 't_test': 0.6593830585479736, 'sg': 1}


In [39]:
model["homme"]

array([-2.68843603e+00,  1.93454778e+00, -1.81924117e+00,  2.36969948e+00,
        9.32004392e-01,  1.25195098e+00,  8.65360320e-01,  3.11363190e-01,
        6.06419086e-01,  1.43122864e+00,  1.65650344e+00, -3.67380810e+00,
        1.87034404e+00,  1.74155509e+00, -5.33656299e-01,  1.40374994e+00,
        2.30875421e+00,  1.79799902e+00, -1.87658596e+00,  1.29379809e+00,
        9.08876181e-01,  1.08965196e-01, -1.40824032e+00,  8.72196436e-01,
        1.08273089e+00,  7.29817152e-01, -1.11542833e+00,  1.52417982e+00,
        1.71380103e+00, -2.30002356e+00,  1.15143669e+00, -2.83064008e-01,
        2.14521900e-01,  8.66827071e-01,  7.54454792e-01,  2.86816597e-01,
        5.41400373e-01, -7.38874137e-01,  5.10816514e-01, -1.25636995e+00,
       -8.32316935e-01,  3.15392661e+00, -6.42400444e-01, -1.62124023e-01,
       -1.09802079e+00,  1.84242070e+00, -3.25098783e-01, -6.75076485e-01,
        1.25538635e+00,  9.76222456e-01, -1.26038420e+00, -8.73921692e-01,
       -4.83060241e-01, -

In [40]:
model.wv.syn0[model.wv.vocab["homme"].index] == model["homme"]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [41]:
model.syn1neg.shape

(50130, 300)

In [42]:
vh = model.wv.vocab["homme"]
vh.index

419